# Drive mount

In [0]:
from google.colab import drive
import os

DRIVE_MOUNT='/content/gdrive'
drive.mount(DRIVE_MOUNT)

IMG_FOLDER=os.path.join('/content/gdrive/My Drive/Code gen for safety contract', 'Images')
os.makedirs(IMG_FOLDER, exist_ok=True)

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/gdrive


# Test running a file from Google Drive

In [0]:
import sys
sys.path.insert(0, '/content/gdrive/My Drive/Code gen for safety contract')

!cp /content/gdrive/My\ Drive/Code\ gen\ for\ safety\ contract/update_library.py /usr/local/bin
!cp /content/gdrive/My\ Drive/Code\ gen\ for\ safety\ contract/lib_obj.py /usr/local/bin
!cp /content/gdrive/My\ Drive/Code\ gen\ for\ safety\ contract/im_obj.py /usr/local/bin
!cp /content/gdrive/My\ Drive/Code\ gen\ for\ safety\ contract/image_generation_utils.py /usr/local/bin 
!chmod 755 /usr/local/bin/update_library.py
!chmod 755 /usr/local/bin/lib_obj.py
!chmod 755 /usr/local/bin/im_obj.py
!chmod 755 /usr/local/bin/image_generation_utils.py
!python /usr/local/bin/update_library.py

In [0]:
from __future__ import print_function
from ipywidgets import interact, interactive, fixed, interact_manual
import ipywidgets as widgets
import cv2
from ipywidgets import FloatSlider

from PIL import Image, ImageFilter
import PIL
from IPython import display

from matplotlib.pyplot import imshow
from matplotlib import pyplot as plt
%matplotlib inline

import pickle

import sys
sys.path.insert(0, '/content/gdrive/My Drive/Code gen for safety contract')

from gen_pic_utils import *
from gen_pic_utils import *
from gen_utils import *
from image_generation_utils import *
from update_library import *

library = update_library()

image_size = (1242,375)

def save(filename,data):
    with open(filename, 'w+') as f:
        pickle.dump(data, f)
        
def load(filename):
    with open(filename, 'rb') as f:
        return pickle.load(f)

# Init variables
poly_verts = 4
polygon = [(image_size[0],image_size[1]),(image_size[0],1),(1,1),(1,image_size[1])]
scales = [1, 1]
    
# Slider parameters
point_widget = FloatSlider(min=0, max=poly_verts-1, step=1, value=0)
x_widget = FloatSlider(min=1, max=image_size[0], step=1, value=image_size[0])
y_widget = FloatSlider(min=1, max=image_size[1], step=0.5, value=image_size[1])
car_widget = FloatSlider(min=0, max=1, step=1, value=0)
scale_widget = FloatSlider(min=0.1, max=2.0, step=0.05, value=1)
car_x_pos_widget = FloatSlider(min=0.0, max=1.0, step=0.001, value=0.5)
car_y_pos_widget = FloatSlider(min=0.0, max=1.0, step=0.01, value=0)

def update_car_conf_widgets(*args):
    (x,y) = polygon[int(point_widget.value)]
    x_widget.value = x
    y_widget.value = y
    
def update_car_scale_widgets(*args):    
    scale = scales[int(car_widget.value)]
    scale_widget.value = scale
    
car_widget.observe(update_car_scale_widgets, 'value')    
point_widget.observe(update_car_conf_widgets, 'value')

def update_point_widgets(point,x,y,car_pos,scale,car_x_pos,car_y_pos,img_name):
    #print(car_pos)
    point_id = int(point_widget.value)
    polygon[point_id] = (x_widget.value,y_widget.value)
    tmp_pt = list(polygon[3 - point_id])
    tmp_pt[1] = y_widget.value
    polygon[3 - point_id] = tuple(tmp_pt)    
        
    polygon[int(point_widget.value)] = (x_widget.value,y_widget.value)
    scales[int(car_pos)] = scale       
    
    library = update_library()

    background = library.background_objects[road_type]
    background_copy = background.image.copy()

    # remove alpha channel from background (if present)
    if background_copy.mode in ('RGBA', 'LA') or \
            (background_copy.mode == 'P' and
                     'transparency' in background_copy.info):
        background_no_alpha = \
            Image.new("RGB", background_copy.size, (255, 255, 255))
        background_no_alpha.paste(background_copy,
                                  mask=background_copy.split()[3])
                                # 3 is the alpha channel
    else:
        background_no_alpha = background_copy
        
    road = background_no_alpha 
    car = library.foreground_objects[car_type].image
    
    #lrx = 1242, lry = 375, llx = 1
    lrx = polygon[0][0]
    lry = polygon[0][1]
    llx = polygon[3][0]
    
    #urx = 1242, ury = 1, ulx = 1
    urx = polygon[1][0]
    ury = polygon[1][1]
    ulx = polygon[2][0]
    
    ####PREVIOUS####
    if(car_pos == 0):
      ##NEW###
      #car_x = int(llx + (lrx-llx)*car_x_pos)
      #####PREVIOUS$}####}####
      car_x = int(llx + (lrx-llx)*car_x_pos - car.size[0]/2)
      #print(car.size[0], car.size[1])
      ######PREVIOUS#######
      #car_y = int(lry - (car.size[1]/2))
      ####NEW####
      car_y = int(lry - (lry-ury)*car_y_pos  - (car.size[1]/2))
      ##NEW###
      # car_y = int(lry)      
        #print(car_x, car_y)
    #####PREVIOUS######
    else:
      car_x = int(ulx + (urx-ulx)*car_x_pos - car.size[0]/2)
      car_y = int(ury - (car.size[1]/2))
    
    ####PREVIOUS######
    #print(car.size[0], ',', car.size[1])
    car = car.resize((int(car.size[0]*scale), int(car.size[1]*scale)), PIL.Image.ANTIALIAS)
    #print("REsized- ", car.size[0], ',', car.size[1])
      #print("Resizing")
    ##NEW###
    #div_fac=15
      #car = car.resize((int((car.size[0]-vert_dist/div_fac)*scale), int((car.size[1]-vert_dist/div_fac)*scale)), PIL.Image.ANTIALIAS) 
    ###NEW###
    #car = car.resize((int((car.size[0]-np.log(car.size[0]*10)*vert_dist/div_fac)*scale), int((car.size[1]-vert_dist/div_fac)*scale)), PIL.Image.ANTIALIAS) 
      #car = car.resize((int((car.size[0]-np.log(car.size[0]*100)*vert_dist/div_fac)*scale), int((car.size[1]-np.log(car.size[1]*100)-vert_dist/div_fac)*scale)), PIL.Image.ANTIALIAS) 
    #print(car.size[0]-np.exp(vert_dist/100)-vert_dist/4,', ', car.size[1]-np.exp(vert_dist/100)-vert_dist/10)
      
    #NEW##
    #road.paste(car, (car_x+int(vert_dist/2),car_y-vert_dist), car)
    #road.paste(car, (car_x,car_y-vert_dist), car)  
    #####PREVIOUS######
    #print(car_x, car_y)
    road.paste(car, (car_x, car_y), car) 
    #road = road.resize((128,128))
    draw = ImageDraw.Draw(road)
    #######Displaying the img as output#####
    # display.display(road)
    #cv2.imshow(road)

    #Saving image in drive
    IMG_SCALE_FOLDER=os.path.join(IMG_FOLDER, 'Scale-'+str(scale))
    os.makedirs(IMG_SCALE_FOLDER, exist_ok=True)
    road.save(os.path.join(IMG_SCALE_FOLDER, img_name), "PNG")

    ####PREVIOUS#########
    #eps = 5
    #draw.ellipse((x-eps, y-eps, x+eps, y+eps), fill = 'blue', outline ='blue')
    #draw.polygon(polygon, fill=None, outline=(255,0,0,255))      
    #return road  

car_id=0
car_type= car_id
road_id=4
road_type=road_id
starting_car_y_pos = 0
starting_scale = 1
starting_car_x_pos = 0.5
for i in range(0, 19):
#i = 17
  car_y_pos = starting_car_y_pos + i*0.01
  scale = starting_scale - 0.05*i
  car_x_pos =  starting_car_x_pos + i*0.006
  for car_id in range(0, 15):
    car_type = car_id
    for road_id in range(4, 49):
      road_type = road_id
      interact(update_point_widgets,point=point_widget, x=x_widget, y=y_widget, car_pos=car_widget, scale=scale, car_x_pos=car_x_pos, car_y_pos=car_y_pos, img_name='car'+str(car_id)+'_'+'road'+str(road_id)+'.png')

 

        



675
